# Figures for Thesis Results

This notebook aggregates pulse outputs from CMA-ES and GRAPE runs, resamples them to a shared time grid, and rescales the pulse areas so that each Omega channel integrates to $4\pi$.


In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import defaultdict
from warnings import warn

In [2]:
BASE_DIR = Path('/home/yehon/projects/grape-crab-qoc')
RESULT_DIRS = {
    'CMA-ES': BASE_DIR / 'notebooks' / 'results' / 'CMA-ES',
    'GRAPE': BASE_DIR / 'notebooks' / 'results' / 'GRAPE',
}
OUTPUT_DIR = BASE_DIR / 'results' / 'results'

CANONICAL_DURATION = 0.1
CANONICAL_POINTS = 2000
CANONICAL_T = np.linspace(0.0, CANONICAL_DURATION, CANONICAL_POINTS)

TARGETS = ['original', 'adiabatic', 'terminal', 'ensemble']
TARGET_ALIASES = {
    'original': ['original', 'baseline'],
    'adiabatic': ['adiabatic'],
    'terminal': ['terminal', 'fidelity'],
    'ensemble': ['ensemble'],
}
CHANNELS = ['omega', 'delta']
CHANNEL_ALIASES = {
    'omega': ['omega'],
    'delta': ['delta'],
}

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
print(f'Artifacts will be written to: {OUTPUT_DIR}')

Artifacts will be written to: /home/yehon/projects/grape-crab-qoc/results/results


In [3]:
def iter_pulse_files(directory: Path):
    for path in directory.glob('*.npy'):
        if path.is_file():
            yield path


def resolve_pulse_file(directory: Path, target: str, channel: str) -> Path:
    target_aliases = [alias.lower() for alias in TARGET_ALIASES[target]]
    channel_aliases = [alias.lower() for alias in CHANNEL_ALIASES[channel]]
    for pulse_path in iter_pulse_files(directory):
        name = pulse_path.name.lower()
        if any(alias in name for alias in target_aliases) and any(alias in name for alias in channel_aliases):
            return pulse_path
    target_desc = ', '.join(target_aliases)
    channel_desc = ', '.join(channel_aliases)
    raise FileNotFoundError(f'No {channel} pulse for target "{target}" (aliases: {target_desc}) in {directory}')


def load_and_resample(path: Path, canonical_t: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    raw = np.load(path)
    if raw.ndim != 1:
        raise ValueError(f'Expected 1D pulse in {path}, received shape {raw.shape}')
    original_t = np.linspace(canonical_t[0], canonical_t[-1], raw.size)
    resampled = np.interp(canonical_t, original_t, raw)
    return raw, resampled


def scale_pulses(omega: np.ndarray, delta: np.ndarray, canonical_t: np.ndarray) -> tuple[float, np.ndarray, np.ndarray, float, float]:
    area = np.trapezoid(omega, canonical_t)
    if np.isclose(area, 0.0):
        warn('Omega area is approximately zero; skipping rescale and leaving pulses unchanged.')
        scale = 1.0
    else:
        scale = 4 * np.pi / area
    omega_scaled = omega * scale
    delta_scaled = delta * scale
    scaled_area = np.trapezoid(omega_scaled, canonical_t)
    return scale, omega_scaled, delta_scaled, area, scaled_area


def slugify(*parts: str) -> str:
    return '_'.join(part.lower().replace('-', '_') for part in parts)


In [4]:
resampled_pulses: dict[str, dict[str, dict[str, np.ndarray]]] = defaultdict(dict)
summary_rows: list[dict[str, float]] = []
missing_entries: list[str] = []

for algorithm, directory in RESULT_DIRS.items():
    if not directory.exists():
        warn(f'Skipping {algorithm}: directory {directory} not found.')
        continue
    for target in TARGETS:
        try:
            omega_path = resolve_pulse_file(directory, target, 'omega')
        except FileNotFoundError as exc:
            missing_entries.append(f'{algorithm} | {target} | omega')
            warn(str(exc))
            continue
        try:
            delta_path = resolve_pulse_file(directory, target, 'delta')
        except FileNotFoundError as exc:
            missing_entries.append(f'{algorithm} | {target} | delta')
            warn(str(exc))
            continue

        _, omega_resampled = load_and_resample(omega_path, CANONICAL_T)
        _, delta_resampled = load_and_resample(delta_path, CANONICAL_T)

        scale, omega_scaled, delta_scaled, area_before, area_after = scale_pulses(
            omega_resampled, delta_resampled, CANONICAL_T
        )

        resampled_pulses[algorithm][target] = {
            'omega': omega_scaled,
            'delta': delta_scaled,
            'scale_factor': scale,
            'area_before': area_before,
            'area_after': area_after,
        }

        summary_rows.append(
            {
                'algorithm': algorithm,
                'target': target,
                'omega_file': omega_path.name,
                'delta_file': delta_path.name,
                'scale_factor': scale,
                'area_before': area_before,
                'area_after': area_after,
            }
        )

summary_df = pd.DataFrame(summary_rows)
summary_df

/tmp/ipykernel_971452/2357453103.py:14: UserWarning: No omega pulse for target "adiabatic" (aliases: adiabatic) in /home/yehon/projects/grape-crab-qoc/notebooks/results/GRAPE
  warn(str(exc))


,algorithm,target,omega_file,delta_file,scale_factor,area_before,area_after
0,CMA-ES,original,Omega_Original.npy,Delta_Original.npy,1.000000,12.566371,12.566371
1,CMA-ES,adiabatic,Omega_Adiabatic.npy,Delta_Adiabatic.npy,1.163252,10.802794,12.566371
2,CMA-ES,terminal,Omega_Fidelity.npy,Delta_Fidelity.npy,1.296217,9.694650,12.566371
3,CMA-ES,ensemble,Omega_Ensemble.npy,Delta_Ensemble.npy,1.064754,11.802130,12.566371
4,GRAPE,original,OMEGA_baseline.npy,DELTA_baseline.npy,1.000000,12.566367,12.566371
5,GRAPE,terminal,OMEGA_terminal.npy,DELTA_terminal.npy,1.023007,12.283759,12.566371
6,GRAPE,ensemble,OMEGA_ensemble.npy,DELTA_ensemble.npy,13.225346,0.950173,12.566371


In [6]:
for algorithm, targets_dict in resampled_pulses.items():
    if not targets_dict:
        continue
    data = {'t': CANONICAL_T}
    for target, pulse_dict in targets_dict.items():
        slug = slugify(target)
        data[f'{slug}_omega'] = pulse_dict['omega']
        data[f'{slug}_delta'] = pulse_dict['delta']
    output_path = OUTPUT_DIR / f'{slugify(algorithm)}_rescaled_pulses.npz'
    np.savez(output_path, **data)
    print(f'Saved {output_path}')

summary_path = OUTPUT_DIR / 'pulse_scaling_summary.csv'
summary_df.to_csv(summary_path, index=False)
print(f'Saved {summary_path}')

if missing_entries:
    print('Missing pulse entries:')
    for item in missing_entries:
        print(f' - {item}')


Saved /home/yehon/projects/grape-crab-qoc/results/results/cma_es_rescaled_pulses.npz
Saved /home/yehon/projects/grape-crab-qoc/results/results/grape_rescaled_pulses.npz
Saved /home/yehon/projects/grape-crab-qoc/results/results/pulse_scaling_summary.csv
Missing pulse entries:
 - GRAPE | adiabatic | omega
